# Equalised Odds post-processing method

In [1]:
import sys
import json
import os
import torch
import pandas as pd
import numpy as np

from confection import Config
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support
from itertools import chain

from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.datasets import BinaryLabelDataset

# update the path so we can directly import code from the DVlog
sys.path.append(os.path.dirname(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "DVlog"))))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "DVlog")))

from DVlog.evaluate import evaluate_model
from DVlog.models.model import UnimodalDVlogModel
from DVlog.utils.dataloaders import MultimodalEmbeddingsDataset
from DVlog.utils.metrics import calculate_performance_measures, calculate_gender_performance_measures, calculate_fairness_measures
from DVlog.utils.util import ConfigDict, validate_config, process_config, set_seed

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[inFairness]'


In [2]:
# Function to apply EqOddsPostprocessing
def apply_eqodds(y_train_true, y_train_pred, y_val_pred, y_test_pred, protected_attr_train, protected_attr_val, protected_attr_test, seed):
    # Create BinaryLabelDataset for training data
    dataset_train_true = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_train_true,
        'protected': protected_attr_train
    }), label_names=['label'], protected_attribute_names=['protected'])

    dataset_train_pred = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_train_pred,
        'protected': protected_attr_train
    }), label_names=['label'], protected_attribute_names=['protected'])
    
    # Create BinaryLabelDataset for val data
    dataset_val_pred = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_val_pred,
        'protected': protected_attr_val
    }), label_names=['label'], protected_attribute_names=['protected'])

    # Create BinaryLabelDataset for test data
    dataset_test_pred = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=pd.DataFrame({
        'label': y_test_pred,
        'protected': protected_attr_test
    }), label_names=['label'], protected_attribute_names=['protected'])

    # Apply EqOddsPostprocessing
    eq_odds = EqOddsPostprocessing(unprivileged_groups=[{'protected': 0}],
                                   privileged_groups=[{'protected': 1}], seed=seed)

    eq_odds = eq_odds.fit(dataset_train_true, dataset_train_pred)
    dataset_transf_train_pred = eq_odds.predict(dataset_train_pred)
    dataset_transf_val_pred = eq_odds.predict(dataset_val_pred)
    dataset_transf_test_pred = eq_odds.predict(dataset_test_pred)

    # Extract the adjusted predictions
    return dataset_transf_train_pred.labels.ravel(), dataset_transf_val_pred.labels.ravel(), dataset_transf_test_pred.labels.ravel()


# build the function for automatically retrieve all metrics
def evaluate_metrics(y_true, y_pred, protected):

    # calculate the performance metrics
    w_precision, w_recall, w_fscore, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

    # calculate the fairness metrics
    eq_oppor, eq_acc, pred_equal, _, _ = calculate_fairness_measures(y_true, y_pred, protected, unprivileged='m')
    
    # eq_oppor, eq_acc, fairl_eq_odds, unpriv_stats, priv_stats = calculate_fairness_measures(y_true, y_pred, protected, 'm')
    gender_metrics = calculate_gender_performance_measures(y_true, y_pred, protected)

    measure_dict = {
        "precision": w_precision,
        "recall": w_recall,
        "fscore": w_fscore,
        f"{gender_metrics[0][0]}_fscore": gender_metrics[0][3],
        f"{gender_metrics[1][0]}_fscore": gender_metrics[1][3],
        "eq_oppor": eq_oppor,
        "eq_acc": eq_acc,
        "pred_eq": pred_equal}
    return measure_dict

# Evaluate unimodal sentence detection models
Load in both the normal and keyword filtered model and evaluate them using the test set in order to retrieve the prediction information

In [3]:
models_path = Path(r"../DVlog/trained_models")
model_config = Path(r"../DVlog/model_configs/unimodal/unimodal_mpnet_sent_keyw.cfg")
annotations_file = Path(r"../DVlog/dataset/dvlog_labels_v2.csv")
save_path = Path(r"../DVlog/trained_models/eqodds_metrics.csv")
device = torch.device('cpu')

random_seeds = [0, 1, 42, 1123, 3407]
results = []

In [4]:
# load in the config file for the model
config = Config().from_disk(model_config)
config_dict = process_config(config)

# overwrite the annotations_file + data_dir
config_dict.annotations_file = annotations_file
config_dict.data_dir = Path("../DVlog/dataset/sent-embeddings-dataset")
config_dict.encoder1_data_dir = Path("../DVlog/dataset/sent-embeddings-dataset")

# setup the model paths
model_dir_path = Path(os.path.join(models_path, config_dict.model_name))

# 
for seed in random_seeds:
    # set the exact model_path
    saved_model_path = Path(os.path.join(model_dir_path, f"model_{config_dict.model_name}_seed{seed}.pth"))

    # et the seed
    set_seed(seed)
    
    # setup the dataset + loader
    train_data = MultimodalEmbeddingsDataset("train", config_dict, to_tensor=True, with_protected=True)
    train_dataloader = DataLoader(train_data, batch_size=config_dict.batch_size, shuffle=True)
    
    val_data = MultimodalEmbeddingsDataset("val", config_dict, to_tensor=True, with_protected=True)
    val_dataloader = DataLoader(val_data, batch_size=config_dict.batch_size, shuffle=True)

    test_data = MultimodalEmbeddingsDataset("test", config_dict, to_tensor=True, with_protected=True)
    test_dataloader = DataLoader(test_data, batch_size=config_dict.batch_size, shuffle=True)

    # setup the model
    saved_model = UnimodalDVlogModel((config_dict.sequence_length, config_dict.encoder1_dim),
                                      d_model=config_dict.dim_model, n_heads=config_dict.uni_n_heads, use_std=config_dict.detectlayer_use_std)

    # load in the parameters and set the model to evaluation mode
    saved_model.load_state_dict(torch.load(saved_model_path, map_location=device))
    saved_model.eval()

    # run the model on the training set
    train_pred, train_y, train_protected, _ = evaluate_model(saved_model, train_dataloader, config_dict, "m", False, seed, True)
    val_pred, val_y, val_protected, _ = evaluate_model(saved_model, val_dataloader, config_dict, "m", False, seed, True)
    test_pred, test_y, test_protected, _ = evaluate_model(saved_model, test_dataloader, config_dict, "m", False, seed, True)

    # reshape all predictions, ground truths, and protected
    train_pred = np.argmax(train_pred, axis=1)
    train_y = np.argmax(train_y, axis=1)
    train_protected_float = np.where(train_protected == "m", 0, 1)
    
    val_pred = np.argmax(val_pred, axis=1)
    val_y = np.argmax(val_y, axis=1)
    val_protected_float = np.where(val_protected == "m", 0, 1)

    test_pred = np.argmax(test_pred, axis=1)
    test_y = np.argmax(test_y, axis=1)
    test_protected_float = np.where(test_protected == "m", 0, 1)

    # run the post-processing code
    new_train_preds, new_val_preds, new_test_preds = apply_eqodds(train_y, train_pred, val_pred, test_pred, train_protected_float, val_protected_float, test_protected_float, seed)

    # evaluate the models
    eval_dict_train = evaluate_metrics(train_y, new_train_preds, train_protected)
    eval_dict_val = evaluate_metrics(val_y, new_val_preds, val_protected)
    eval_dict_test = evaluate_metrics(test_y, new_test_preds, test_protected)

    results.append(("eqodds", "train", seed, eval_dict_train))
    results.append(("eqodds", "val", seed, eval_dict_val))
    results.append(("eqodds", "test", seed, eval_dict_test))

Model: unimodal_mpnet_sent_keyw with seed: 0
----------
Model: unimodal_mpnet_sent_keyw with seed: 0
----------
Model: unimodal_mpnet_sent_keyw with seed: 0
----------
Model: unimodal_mpnet_sent_keyw with seed: 1
----------
Model: unimodal_mpnet_sent_keyw with seed: 1
----------
Model: unimodal_mpnet_sent_keyw with seed: 1
----------
Model: unimodal_mpnet_sent_keyw with seed: 42
----------
Model: unimodal_mpnet_sent_keyw with seed: 42
----------
Model: unimodal_mpnet_sent_keyw with seed: 42
----------
Model: unimodal_mpnet_sent_keyw with seed: 1123
----------
Model: unimodal_mpnet_sent_keyw with seed: 1123
----------
Model: unimodal_mpnet_sent_keyw with seed: 1123
----------
Model: unimodal_mpnet_sent_keyw with seed: 3407
----------
Model: unimodal_mpnet_sent_keyw with seed: 3407
----------
Model: unimodal_mpnet_sent_keyw with seed: 3407
----------


In [5]:
# Extract data into a structured format
extracted_data = []
for name, dataset, seed, result in results:
    data = {
        "name": name,
        "dataset": dataset,
        "seed": seed,
        "precision": np.round(result["precision"], 3),
        "recall": np.round(result["recall"], 3),
        "f1": np.round(result["fscore"], 3),
        "f1_m": np.round(result["m_fscore"], 3),
        "f1_f": np.round(result["f_fscore"], 3),
        "eq opportunity": np.round(result["eq_oppor"], 2),
        "Eq accuracy": np.round(result["eq_acc"], 2),
        "pred equality": np.round(result["pred_eq"], 2)
    }
    extracted_data.append(data)

# Convert the list of dictionaries to a pandas DataFrame and display it
df = pd.DataFrame(extracted_data)
df.groupby(["dataset", "name"]).mean()

,,seed,precision,recall,f1,f1_m,f1_f,eq opportunity,Eq accuracy,pred equality
dataset,name,,,,,,,,,
test,eqodds,914.6,0.9270,0.9258,0.9256,0.8806,0.9500,0.896,0.928,1.488
train,eqodds,914.6,0.9762,0.9760,0.9758,0.9770,0.9752,1.000,1.000,0.966
val,eqodds,914.6,0.9292,0.9286,0.9286,0.9310,0.9270,0.988,1.002,1.028


In [6]:
# df.to_csv(save_path, sep=";")